### Define necessary functions 

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ast
import json
import csv
import os

def load_log_data(exp_dir):
    file_path = os.path.join(exp_dir, 'objectnav-dino/logs/all_info.log')
    data_dict = {}
    data_list = []
    with open(file_path, "r") as file:
        for line in file:
            line_dict = json.loads(line.strip())
            data_dict[line_dict['episode']] = line_dict
            data_list.append(line_dict)
    return data_dict, data_list

def load_from_csv(filename):
    data = []
    skip_row_count = 0
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Skip header row
        for row in reader:
            if len(row[0]) != 0:
                data.append(row)  # there need to be something in the first column
            else:
                skip_row_count += 1
    print(f"Skipped {skip_row_count} empty rows.")
        # data = [row for row in reader]
    return data, header

def load_from_csv_list_of_dict(filename):
    data = []
    skip_row_count = 0
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Skip header row
        for row in reader:
            if len(row[0]) != 0:
                row_dict = {}
                for i, key in enumerate(header):
                    row_dict[key] = row[i]
                data.append(row_dict)  # there need to be something in the first column
            else:
                skip_row_count += 1
    print(f"Skipped {skip_row_count} empty rows.")
        # data = [row for row in reader]
    return data, header

def load_from_csv_dict_of_dict(filename):
    data = {}
    skip_row_count = 0
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Skip header row
        for row in reader:
            if len(row[0]) != 0:
                row_dict = {}
                for i, key in enumerate(header):
                    row_dict[key] = row[i]
                data[row[0]] = row_dict # there need to be something in the first column
            else:
                skip_row_count += 1
    print(f"Skipped {skip_row_count} empty rows.")
        # data = [row for row in reader]
    return data, header

def write_list_of_lists_to_csv(data, file_name, header=None):
    with open(file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        if header:
            writer.writerow(header) # optionally write the header
        writer.writerows(data)

def write_list_of_dict_to_csv(data, file_name):
    fieldnames = data[0].keys()
    with open(file_name, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()      # optional: writes the header row
        writer.writerows(data)    # writes all rows

def write_dict_of_dict_to_csv(data, file_name):
    # convert dict of dict to list of dict 
    data_list = []
    for key, value in data.items():
        data_list.append(value)
    write_list_of_dict_to_csv(data_list, file_name)

def construct_server_video_path(server_path, video_path_local):
    file_path = os.path.join(*video_path_local.split('/')[1:]) # ignore the first part, which is dump/
    return os.path.join(server_path, file_path)

def merge_results_to_csv(csv_list, data_dict, header, new_header, video_dir, server_path):
    header += new_header
    effective_data_count = 0
    for csv_data in csv_list:
        episode = csv_data[0]
        if episode in data_dict.keys():
            effective_data_count += 1
            csv_data.append(data_dict[episode]['habitat_success'])
            video_path = os.path.join(video_dir, f"eps_{episode}_vis.mp4")
            csv_data.append(construct_server_video_path(server_path, video_path))
        else:
            # print(f"Episode {episode} not found in log data.")
            csv_data += ['', '']
    print(f"Effective data count: {effective_data_count}/ {len(csv_list)}")
    
    return csv_list, header

#### define universal varialbes

In [8]:
server_path = 'http://goldeen.engin.umich.edu:8820/'

### find diff

In [51]:
exp_dir1 = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair'
# exp_dir2 = 'dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal_struggle2'
exp_dir2 ='dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal_revert_full'

vid_path1 = f'{exp_dir1}/objectnav-dino/episodes_video'
vid_path2 = f'{exp_dir2}/objectnav-dino/episodes_video'

data_dict1, data_list1 = load_log_data(exp_dir1)
data_dict2, data_list2 = load_log_data(exp_dir2)
print(f"Data dict 1 length: {len(data_dict1)}")
print(f"Data dict 2 length: {len(data_dict2)}")

better1 = 0
better2 = 0
better_episode_labels1 = []
better_episode_labels2 = []

# 1 better than 2 episodes
for episode in data_dict1.keys():
    try:
        if data_dict1[episode]['habitat_success'] > data_dict2[episode]['habitat_success']:
            better_episode_labels1.append(episode)
            better1 += 1
    except KeyError:
        continue

# 2 better than 1 episodes
for episode in data_dict2.keys():
    try:
        if data_dict2[episode]['habitat_success'] > data_dict1[episode]['habitat_success']:
            better_episode_labels2.append(episode)
            better2 += 1
    except KeyError:
        continue
        
print(f"Better 1: {better1}")
print(f"Better 2: {better2}")
print(f"better1_episode_labels={better_episode_labels1}")
print(f"better2_episode_labels={better_episode_labels2}")

print(f"Better in {exp_dir1}:\n")
for episode_label in better_episode_labels1:
    vid_name = f'eps_{episode_label}_vis.mp4'
    print(os.path.abspath(os.path.join(vid_path1, vid_name)))
    print(os.path.abspath(os.path.join(vid_path2, vid_name)))
    # print(os.path.join(online_vid_path1, vid_name))
    # print(os.path.join(online_vid_path2, vid_name))
    print('')

print(f"Better in {exp_dir2}:\n")
for episode_label in better_episode_labels2:
    vid_name = f'eps_{episode_label}_vis.mp4'
    print(os.path.abspath(os.path.join(vid_path1, vid_name)))
    print(os.path.abspath(os.path.join(vid_path2, vid_name)))
    # print(os.path.join(online_vid_path1, vid_name))
    # print(os.path.join(online_vid_path2, vid_name))
    print('')

Data dict 1 length: 400
Data dict 2 length: 400
Better 1: 19
Better 2: 32
better1_episode_labels=['bxsVRursffK_35', 'TEEsavR23oF_11', '6s7QHgap2fW_98', 'q3zU7Yy5E5s_80', 'qyAac8rV8Zk_30', 'wcojb4TFT35_8', 'svBbv1Pavdk_65', 'qyAac8rV8Zk_34', 'wcojb4TFT35_92', 'wcojb4TFT35_95', 'ziup5kvtCCR_32', 'mL8ThkuaVTM_82', 'mL8ThkuaVTM_19', 'zt1RVoi7PcG_63', 'qyAac8rV8Zk_81', 'wcojb4TFT35_73', 'qyAac8rV8Zk_86', 'zt1RVoi7PcG_42', 'wcojb4TFT35_56']
better2_episode_labels=['4ok3usBNeis_37', '5cdEh9F2hJL_0', '4ok3usBNeis_43', '5cdEh9F2hJL_81', '5cdEh9F2hJL_65', 'DYehNKdT76V_62', '5cdEh9F2hJL_75', 'TEEsavR23oF_54', 'mv2HUxq3B53_42', 'bxsVRursffK_4', 'bxsVRursffK_7', 'bxsVRursffK_50', 'QaLdnwvtxbs_10', 'qyAac8rV8Zk_4', 'cvZr5TUy5C5_26', 'bxsVRursffK_29', 'XB4GS9ShBRE_98', 'TEEsavR23oF_78', 'cvZr5TUy5C5_3', 'p53SfW6mjZe_5', 'zt1RVoi7PcG_78', 'wcojb4TFT35_22', 'qyAac8rV8Zk_41', 'mL8ThkuaVTM_24', 'p53SfW6mjZe_59', 'qyAac8rV8Zk_7', 'mv2HUxq3B53_23', 'qyAac8rV8Zk_56', 'svBbv1Pavdk_28', 'svBbv1Pavdk_20', 'qyA

In [49]:
# calculate the average success rate and spl
exp_dir = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair'
data_dict, data_list = load_log_data(exp_dir2)
num_success = 0
spl_list = []
for i, itmes, in enumerate(data_dict.items()):
    key, item = itmes
    if item['habitat_success'] > 0:
        num_success += 1
        spl_list.append(item['spl'])
avg_spl = np.sum(spl_list) / len(data_dict)
avg_success_rate = num_success / len(data_dict)
print(f"Average success rate: {num_success}/{len(data_dict)} {avg_success_rate}")
print(f"Average spl: {avg_spl}")

Average success rate: 288/400 0.72
Average spl: 0.3812233465953423


### save log as csv with video 

In [4]:
exp_dir = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal'
video_dir = os.path.join(exp_dir, 'objectnav-dino/episodes_video')
data_dict, data_list = load_log_data(exp_dir)
print(f"Loaded {len(data_list)} data points from log file.")
print(f"Loaded {len(data_dict)} data points from log file.")

# append video path to data_dict 
for episode in data_dict.keys():
    video_path = os.path.join(video_dir, f"eps_{episode}_vis.mp4")
    data_dict[episode]['video_path'] = construct_server_video_path(server_path, video_path)

# save the data_dict to a csv file
write_dict_of_dict_to_csv(data_dict, 'experiments/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal.csv')

Loaded 400 data points from log file.
Loaded 400 data points from log file.


### merge new results in existing csv

In [18]:
# load experiment log
# exp_dir = 'dump/apr21_hm3d_baseline_stair'
exp_dir = 'dump/apr21_baseline_gt_perception_stair'
video_dir = os.path.join(exp_dir, 'objectnav-dino/episodes_video')
# log_file = os.path.join(exp_dir, 'objectnav-dino/logs/all_info.log')
data_dict, data_list = load_log_data(exp_dir)
print(data_dict.keys())

dict_keys(['4ok3usBNeis_69', 'cvZr5TUy5C5_18', 'Nfvxx8J5NCo_6', 'TEEsavR23oF_3', '4ok3usBNeis_37', 'mL8ThkuaVTM_91', 'mL8ThkuaVTM_86', '4ok3usBNeis_12', 'zt1RVoi7PcG_78', 'qyAac8rV8Zk_12', 'mL8ThkuaVTM_56', 'QaLdnwvtxbs_96', '4ok3usBNeis_62', 'qyAac8rV8Zk_91', 'XB4GS9ShBRE_17', 'QaLdnwvtxbs_77', '4ok3usBNeis_95', 'zt1RVoi7PcG_116', '4ok3usBNeis_16', '5cdEh9F2hJL_80', '6s7QHgap2fW_94', 'Dd4bFSTQ8gi_75', 'q3zU7Yy5E5s_22', '6s7QHgap2fW_37', 'XB4GS9ShBRE_53', 'p53SfW6mjZe_87', 'mv2HUxq3B53_88', '5cdEh9F2hJL_76', 'Nfvxx8J5NCo_98', 'mL8ThkuaVTM_28', 'bxsVRursffK_30', 'qyAac8rV8Zk_45', 'ziup5kvtCCR_77', 'q3zU7Yy5E5s_41', 'XB4GS9ShBRE_72', 'DYehNKdT76V_97', 'ziup5kvtCCR_80', 'p53SfW6mjZe_37', 'mL8ThkuaVTM_59', 'XB4GS9ShBRE_88', 'ziup5kvtCCR_64', 'mv2HUxq3B53_28', 'DYehNKdT76V_62', 'qyAac8rV8Zk_41', 'wcojb4TFT35_17', 'Nfvxx8J5NCo_5', 'XB4GS9ShBRE_9', 'mL8ThkuaVTM_60', '6s7QHgap2fW_80', 'zt1RVoi7PcG_113', 'ziup5kvtCCR_0', 'XB4GS9ShBRE_11', 'svBbv1Pavdk_40', 'qyAac8rV8Zk_2', 'mv2HUxq3B53_16', '6s

In [19]:
csv_file = 'experiments/gt perception result analysis - result alphbet.csv'

csv_list, header = load_from_csv(csv_file)
print(csv_list)
print(header)

Skipped 605 empty rows.
[['4ok3usBNeis_63', '0', 'D', '', 'http://goldeen.engin.umich.edu:8000/eps_4ok3usBNeis_63_vis.mp4', '4ok3usBNeis_63', '0.994019568', '0', '9', '0', '1', '4', 'collision', '0', '0', 'TRUE', 'FALSE', '129', 'sofa'], ['5cdEh9F2hJL_36', '0', 'I', '', 'http://goldeen.engin.umich.edu:8000/eps_5cdEh9F2hJL_36_vis.mp4', '5cdEh9F2hJL_36', '0.124308534', '0', '108', '8', '4', '38', 'collision', '0', '0', 'FALSE', 'FALSE', '155', 'toilet'], ['DYehNKdT76V_63', '0', 'D', '', 'http://goldeen.engin.umich.edu:8000/eps_DYehNKdT76V_63_vis.mp4', 'DYehNKdT76V_63', '4.79305172', '0', '73', '5', '2', '26', 'collision', '0', '0', 'FALSE', 'FALSE', '47', 'sofa'], ['p53SfW6mjZe_44', '0', 'D', '', 'http://goldeen.engin.umich.edu:8000/eps_p53SfW6mjZe_44_vis.mp4', 'p53SfW6mjZe_44', '2.768301249', '0', '173', '25', '8', '55', 'collision', '0', '0', 'FALSE', 'FALSE', '38', 'tv_monitor'], ['q3zU7Yy5E5s_80', '0', 'G', '', 'http://goldeen.engin.umich.edu:8000/eps_q3zU7Yy5E5s_80_vis.mp4', 'q3zU7Y

In [20]:
# add new episode info to the csv according to the csv info 
csv_list, header = merge_results_to_csv(csv_list, data_dict, header, ['new_success', 'new_video_link'], video_dir, server_path)

Effective data count: 164/ 164


In [21]:
# write the new csv file
csv_file = 'experiments/old_failure_vs_new_pipline.csv'
write_list_of_lists_to_csv(csv_list, csv_file, header=header)

In [22]:
episode_labels = ['q3zU7Yy5E5s_22', 'wcojb4TFT35_17', 'Nfvxx8J5NCo_5', 'svBbv1Pavdk_28', 'ziup5kvtCCR_30', 'svBbv1Pavdk_9', 'Nfvxx8J5NCo_13', 'qyAac8rV8Zk_85', 'wcojb4TFT35_22', 'wcojb4TFT35_41', 'bxsVRursffK_49', 'TEEsavR23oF_40', 'wcojb4TFT35_48', '5cdEh9F2hJL_75', 'wcojb4TFT35_32', '5cdEh9F2hJL_34', 'TEEsavR23oF_88', 'ziup5kvtCCR_49', 'bxsVRursffK_72', '5cdEh9F2hJL_61', 'Nfvxx8J5NCo_68', 'zt1RVoi7PcG_94', 'qyAac8rV8Zk_16', 'Dd4bFSTQ8gi_69', 'cvZr5TUy5C5_61', 'bxsVRursffK_64']

old_vidoe_dir = 'dump/apr15_baseline_gt_perception_stair/objectnav-dino/episodes_video'
old_better_list = []
for episode in episode_labels:
    row = [episode]
    new_video_path = os.path.join(video_dir, f"eps_{episode}_vis.mp4")
    old_video_path = os.path.join(old_vidoe_dir, f"eps_{episode}_vis.mp4")

    row.append(construct_server_video_path(server_path, old_video_path))
    row.append(construct_server_video_path(server_path, new_video_path))
    old_better_list.append(row)

header = ['episode', 'old_video_link', 'new_video_link']
write_list_of_lists_to_csv(old_better_list, 'experiments/old_better_list.csv', header=header)

### write "better in" results as diff in single csv

In [52]:
better1_episode_labels=['bxsVRursffK_35', 'TEEsavR23oF_11', '6s7QHgap2fW_98', 'q3zU7Yy5E5s_80', 'qyAac8rV8Zk_30', 'wcojb4TFT35_8', 'svBbv1Pavdk_65', 'qyAac8rV8Zk_34', 'wcojb4TFT35_92', 'wcojb4TFT35_95', 'ziup5kvtCCR_32', 'mL8ThkuaVTM_82', 'mL8ThkuaVTM_19', 'zt1RVoi7PcG_63', 'qyAac8rV8Zk_81', 'wcojb4TFT35_73', 'qyAac8rV8Zk_86', 'zt1RVoi7PcG_42', 'wcojb4TFT35_56']


better2_episode_labels=['4ok3usBNeis_37', '5cdEh9F2hJL_0', '4ok3usBNeis_43', '5cdEh9F2hJL_81', '5cdEh9F2hJL_65', 'DYehNKdT76V_62', '5cdEh9F2hJL_75', 'TEEsavR23oF_54', 'mv2HUxq3B53_42', 'bxsVRursffK_4', 'bxsVRursffK_7', 'bxsVRursffK_50', 'QaLdnwvtxbs_10', 'qyAac8rV8Zk_4', 'cvZr5TUy5C5_26', 'bxsVRursffK_29', 'XB4GS9ShBRE_98', 'TEEsavR23oF_78', 'cvZr5TUy5C5_3', 'p53SfW6mjZe_5', 'zt1RVoi7PcG_78', 'wcojb4TFT35_22', 'qyAac8rV8Zk_41', 'mL8ThkuaVTM_24', 'p53SfW6mjZe_59', 'qyAac8rV8Zk_7', 'mv2HUxq3B53_23', 'qyAac8rV8Zk_56', 'svBbv1Pavdk_28', 'svBbv1Pavdk_20', 'qyAac8rV8Zk_37', 'zt1RVoi7PcG_29']


episode_labels = better1_episode_labels + better2_episode_labels

# episode_labels=['5cdEh9F2hJL_61', 'TEEsavR23oF_2', 'qyAac8rV8Zk_86', '5cdEh9F2hJL_82', 'qyAac8rV8Zk_33', 'cvZr5TUy5C5_65', 'qyAac8rV8Zk_56', 'TEEsavR23oF_46', 'Nfvxx8J5NCo_40', '6s7QHgap2fW_82', 'qyAac8rV8Zk_37', '6s7QHgap2fW_44', 'mv2HUxq3B53_98', 'wcojb4TFT35_56', 'svBbv1Pavdk_86', 'qyAac8rV8Zk_4', 'Nfvxx8J5NCo_68', 'zt1RVoi7PcG_75', 'bxsVRursffK_67', 'svBbv1Pavdk_43', 'cvZr5TUy5C5_26', 'XB4GS9ShBRE_62', 'DYehNKdT76V_74', 'mv2HUxq3B53_12', 'Nfvxx8J5NCo_7', 'wcojb4TFT35_32', 'DYehNKdT76V_0', 'XB4GS9ShBRE_69', '6s7QHgap2fW_62', '6s7QHgap2fW_98', 'TEEsavR23oF_61', 'XB4GS9ShBRE_20', 'mL8ThkuaVTM_76', 'wcojb4TFT35_95', 'wcojb4TFT35_94', 'ziup5kvtCCR_0', 'ziup5kvtCCR_32', '5cdEh9F2hJL_72', 'TEEsavR23oF_11', '4ok3usBNeis_12', 'mv2HUxq3B53_23', '5cdEh9F2hJL_0', '5cdEh9F2hJL_6', 'bxsVRursffK_72', 'TEEsavR23oF_50', 'TEEsavR23oF_47', 'TEEsavR23oF_1', 'q3zU7Yy5E5s_60', 'mv2HUxq3B53_64', 'bxsVRursffK_7', 'q3zU7Yy5E5s_58', 'bxsVRursffK_18', 'qyAac8rV8Zk_85', 'q3zU7Yy5E5s_3', 'XB4GS9ShBRE_73', 'zt1RVoi7PcG_0', 'qyAac8rV8Zk_16', '5cdEh9F2hJL_81', 'qyAac8rV8Zk_30', 'bxsVRursffK_64', 'bxsVRursffK_30', 'wcojb4TFT35_15', 'bxsVRursffK_74', 'Nfvxx8J5NCo_6', 'DYehNKdT76V_97', 'mv2HUxq3B53_55', 'qyAac8rV8Zk_87', 'DYehNKdT76V_85', 'cvZr5TUy5C5_90', 'svBbv1Pavdk_55', 'TEEsavR23oF_40', 'DYehNKdT76V_62', '6s7QHgap2fW_17', '6s7QHgap2fW_53', 'XB4GS9ShBRE_7', 'svBbv1Pavdk_38', 'wcojb4TFT35_1']

# dir1 = 'dump/apr27_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred'
# dir2 = 'dump/apr27_hm3d_Ours_RegionOnly_ExploitOnly_GTSGPred'

exp_dir1 = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair'
exp_dir2 ='dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal_revert_full'

data_dict1, data_list1 = load_log_data(exp_dir1)
data_dict2, data_list2 = load_log_data(exp_dir2)

video_subdir = 'objectnav-dino/episodes_video'
video_dir1 = os.path.join(exp_dir1, video_subdir)
video_dir2 = os.path.join(exp_dir2, video_subdir)
csv_list = []
for episode in episode_labels:
    row = [episode]
    old_video_path = os.path.join(video_dir1, f"eps_{episode}_vis.mp4")
    new_video_path = os.path.join(video_dir2, f"eps_{episode}_vis.mp4")

    row.append(data_dict1[episode]['target'])
    row.append(data_dict1[episode]['habitat_success'])
    row.append(construct_server_video_path(server_path, old_video_path))
    row.append(data_dict2[episode]['habitat_success'])
    row.append(construct_server_video_path(server_path, new_video_path))
    csv_list.append(row)

header = ['episode', 'target', 'obj_success', 'obj_video_link', 'region_success', 'region_video_link']
write_list_of_lists_to_csv(csv_list, 'experiments/final_pipeline.csv', header=header)


In [26]:
print(csv_list[-1])

['5cdEh9F2hJL_21', 0.0, 'http://goldeen.engin.umich.edu:8820/apr21_hm3d_OursBaseline_stair/objectnav-dino/episodes_video/eps_5cdEh9F2hJL_21_vis.mp4', 1.0, 'http://goldeen.engin.umich.edu:8820/apr21_hm3d_baseline_stair/objectnav-dino/episodes_video/eps_5cdEh9F2hJL_21_vis.mp4']


### Convert log data to csv with video link

In [9]:
exp_dir = 'dump/apr21_baseline_gt_perception_stair'
data_dict, data_list = load_log_data(exp_dir)
# append video link to the data_list
video_dir = os.path.join(exp_dir, 'objectnav-dino/episodes_video')
for i in range(len(data_list)):
    episode = data_list[i]['episode']
    video_path = os.path.join(video_dir, f"eps_{episode}_vis.mp4")
    data_list[i]['video_link'] = construct_server_video_path(server_path, video_path)
write_list_of_dict_to_csv(data_list, 'experiments/apr21_baseline_gt_perception_stair.csv')

### merge log results (either csv or log file)

In [ ]:
path = '/root/Projects/SG-VLN/experiments/gt perception result analysis -  result ORIGINAL.csv'
csv_dict, header = load_from_csv_dict_of_dict(path) # addition (right side)
print(csv_dict)
print(header)

exp_dir = 'dump/apr21_baseline_gt_perception_stair'
data_dict, data_list = load_log_data(exp_dir) # base (left side)

# merge two list of dict, right side to left side
video_dir = os.path.join(exp_dir, 'objectnav-dino/episodes_video')
for i in range(len(data_list)):
    episode = data_list[i]['episode']
    video_path = os.path.join(video_dir, f"eps_{episode}_vis.mp4")
    data_list[i]['video_link'] = construct_server_video_path(server_path, video_path)
    if episode in csv_dict.keys():
        data_list[i]['old_video_link'] = csv_dict[episode]['video_link']
        data_list[i]['old_success'] = csv_dict[episode]['habitat_success']
        data_list[i]['old_human_analysis'] = csv_dict[episode]['[human] fail category']
        data_list[i]['old_human_notes'] = csv_dict[episode]['[human] notes']
    else:
        data_list[i]['old_video_link'] = 'http://goldeen.engin.umich.edu:8000/' + f"eps_{episode}_vis.mp4"
        data_list[i]['old_success'] = ''
        data_list[i]['old_human_analysis'] = ''
        data_list[i]['old_human_notes'] = ''


write_list_of_dict_to_csv(data_list, 'experiments/old_vs_new_full.csv')

Skipped 605 empty rows.
{'4ok3usBNeis_37': {'episode': '4ok3usBNeis_37', 'habitat_success': '4', '[human] fail category': '4', '[human] notes': '', 'video_link': 'http://goldeen.engin.umich.edu:8000/eps_4ok3usBNeis_37_vis.mp4', 'distance_to_goal': '11.37474823', 'spl': '0', 'success': '156', 'exploration': '15', 'collision': '6', 'detection': '49', 'fail_category': 'exploration', 'switch_upstair_count': '0', 'switch_downstair_count': '0', 'upstair_flag': 'FALSE', 'downstair_flag': 'TRUE', 'count_steps': '500', 'target': 'tv_monitor'}, '4ok3usBNeis_63': {'episode': '4ok3usBNeis_63', 'habitat_success': '0', '[human] fail category': '4', '[human] notes': '', 'video_link': 'http://goldeen.engin.umich.edu:8000/eps_4ok3usBNeis_63_vis.mp4', 'distance_to_goal': '0.994019568', 'spl': '0', 'success': '9', 'exploration': '0', 'collision': '1', 'detection': '4', 'fail_category': 'collision', 'switch_upstair_count': '0', 'switch_downstair_count': '0', 'upstair_flag': 'TRUE', 'downstair_flag': 'FALS

In [9]:
# load base dict 
merge_in_path = 'dump/apr24_hm3d_Ours_GT_SG_stair'
merge_in_dict, merge_in_list = load_log_data(merge_in_path)
# csv_dict, header = load_from_csv_dict_of_dict(merg_in_path) # addition (right side)
merge_in_video_dir = os.path.join(merge_in_path, 'objectnav-dino/episodes_video')

# load merge in dict
# exp_dir = 'dump/apr21_baseline_gt_perception_stair'
base_path = 'experiments/gt perception result analysis - old_vs_new_full.csv'
base_dict, base_header = load_from_csv_dict_of_dict(base_path) # addition (right side)
print(base_header)
print(len(base_dict.keys()))
# data_dict, data_list = load_log_data(exp_dir) # base (left side)

merge_in_prefix = "GT_SG "
# merge two dict of dict, merge_in to base
for episode, entry in base_dict.items():
    video_path = os.path.join(merge_in_video_dir, f"eps_{episode}_vis.mp4")
    if episode in merge_in_dict.keys():
        entry[merge_in_prefix + 'video_link'] = construct_server_video_path(server_path, video_path)
        merge_in_entry = merge_in_dict[episode]
        for key in merge_in_entry.keys():
            entry[merge_in_prefix + key] = merge_in_entry[key]
    else:
        entry[merge_in_prefix + 'video_link'] = ''
        dummy_entry = merge_in_list[0]
        for key in dummy_entry.keys():
            entry[merge_in_prefix + key] = ''
            
data_list = []
for episode, entry in base_dict.items():
    data_list.append(entry)
write_list_of_dict_to_csv(data_list, 'experiments/old_new_GTSG.csv')

Skipped 0 empty rows.
['episode', 'old_success', 'old_human_analysis', 'old_human_notes', 'old_video_link', 'habitat_success', 'distance_to_goal', 'direct cause', 'other problems', 'video_link', 'spl', 'success', 'exploration', 'collision', 'detection', 'fail_category', 'switch_upstair_count', 'switch_downstair_count', 'upstair_flag', 'downstair_flag', 'count_steps', 'target', 'Column 1', 'Column 2', 'Column 3', 'Column 4', 'Column 5', 'Column 6']
400


### remove all success cases

In [7]:
# exp_dir1 = 'dump/apr28_baseline_gt_perception_stair'
# exp_dir2 = 'dump/apr27_baseline_gt_perception_stair'
# exp_dir3 = 'dump/apr27_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred'
# exp_dir4 = 'dump/apr27_hm3d_Ours_ObjectRegion_ExploitOnly_GTSGPred'
# exp_dir5 = 'dump/apr27_hm3d_Ours_RegionOnly_ExploitOnly_GTSGPred'
# exp_dir6 = 'dump/apr26_hm3d_ExploitOnly_ObservedGT_TextScore_stair'
# exp_dir7 = 'dump/apr26_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred'

# dir_list = [exp_dir1, exp_dir2, exp_dir3, exp_dir4, exp_dir5, exp_dir6, exp_dir7]

exp_dir1 = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair'
exp_dir2 = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal'
exp_dir3 = 'dump/apr29_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn'
exp_dir4 = 'dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal2'
exp_dir5 = 'dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal2'
exp_dir6 = 'dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal_struggle'
exp_dir7 = 'dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_GTSGPred_stair_StairSpawn_EdgeGoal_struggle_debug'

dir_list = [exp_dir1, exp_dir2, exp_dir3, exp_dir4, exp_dir5, exp_dir6, exp_dir7]

exp_dict_list = []
for exp_dir in dir_list:
    data_dict, data_list = load_log_data(exp_dir)
    # print(f"Loaded {len(data_list)} data points from log file.")
    print(f"Loaded {len(data_dict)} data points from log file.")
    exp_dict_list.append(data_dict)

episodes = list(exp_dict_list[0].keys())
print(len(episodes))

filtered_episodes = []
for episode in episodes:
    # if all exp sucess, skip this episode
    if all(exp_dict_list[i].get(episode, {'habitat_success': 1})['habitat_success'] == 1 for i in range(len(exp_dict_list))):
        continue
    filtered_episodes.append(episode)
    
print(len(filtered_episodes))
print(filtered_episodes)

Loaded 400 data points from log file.
Loaded 400 data points from log file.
Loaded 369 data points from log file.
Loaded 400 data points from log file.
Loaded 400 data points from log file.
Loaded 315 data points from log file.
Loaded 23 data points from log file.
400
167
['4ok3usBNeis_43', '4ok3usBNeis_37', '6s7QHgap2fW_37', '6s7QHgap2fW_15', '6s7QHgap2fW_94', 'DYehNKdT76V_63', '5cdEh9F2hJL_0', '5cdEh9F2hJL_81', '6s7QHgap2fW_17', 'Nfvxx8J5NCo_98', 'Dd4bFSTQ8gi_75', '5cdEh9F2hJL_65', '6s7QHgap2fW_68', 'DYehNKdT76V_44', 'Nfvxx8J5NCo_66', 'TEEsavR23oF_38', '5cdEh9F2hJL_75', '6s7QHgap2fW_61', 'Nfvxx8J5NCo_5', 'XB4GS9ShBRE_83', '6s7QHgap2fW_84', 'TEEsavR23oF_54', 'DYehNKdT76V_18', 'Nfvxx8J5NCo_68', 'bxsVRursffK_35', 'TEEsavR23oF_64', 'TEEsavR23oF_1', '5cdEh9F2hJL_72', '6s7QHgap2fW_34', 'TEEsavR23oF_5', 'TEEsavR23oF_11', 'DYehNKdT76V_85', 'TEEsavR23oF_43', '6s7QHgap2fW_73', 'XB4GS9ShBRE_31', 'XB4GS9ShBRE_69', 'DYehNKdT76V_74', 'Nfvxx8J5NCo_26', 'Nfvxx8J5NCo_7', 'QaLdnwvtxbs_10', 'bxsVRursff